# La 3-upla de palabras más común en los comentarios de los logs (⭐⭐)

In [2]:
import pyspark as sp
import nltk
import re

In [ ]:
spark = sp.sql.SparkSession.builder.getOrCreate()
rdd = spark.read.csv('../data/logs.csv', header=True, inferSchema=True).rdd

In [4]:
# [(0, 'item_id'),
#  (1, 'timestamp'),
#  (2, 'contributor_username'),
#  (3, 'contributor_id'),
#  (4, 'contributor_ip'),
#  (5, 'comment'),
#  (6, 'logtype'),
#  (7, 'action'),
#  (8, 'title')]

In [6]:
# i)   Me quedo solo con los comentarios que no son nulos y les saco la puntuacion.
# ii)  Tokenizo y me quedo con las 3-uplas.
# iii) Paso los registros a pares (3-upla, 1).
# iv)  Sumo los valores de cada trigrama.
# v)   Me quedo con la 3-upla mas comun.
rdd.filter(lambda x: x[5]).map(lambda x: re.sub(r'[^\w\s]',' ', x[5]))\
    .flatMap(lambda x: nltk.trigrams(nltk.word_tokenize(x)))\
    .map(lambda x: (x, 1))\
    .reduceByKey(lambda x,y: x+y)\
    .reduce(lambda x,y: x if x[1] > y[1] else y)


(('el', 'contenido', 'era'), 1134016)